In [1]:
import pandas as pd

In [3]:
a=pd.read_csv('new_viking_catalog.deduped.csv',on_bad_lines='skip')

In [8]:
!echo $OPENAPI_API_KEY

In [9]:
a.columns

Index(['item_id', 'description', 'manufacturer_name'], dtype='object')

In [24]:
from openai import OpenAI

In [12]:
model='text-embedding-3-small'

In [80]:
import os
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [21]:
a.isna().sum()

item_id                 0
description             0
manufacturer_name    6049
dtype: int64

In [25]:
client=OpenAI(api_key=OPENAI_API_KEY)

In [41]:
def embed(lst):
    response=client.embeddings.create(
        model=model,
        input=lst,
    )
    return [z.embedding for z in response.data]

In [42]:
b=embed(['a','b','c'])

In [45]:
pd.DataFrame(b).shape

(3, 1536)

In [49]:
chsiz=16
i=0
all_embs=[]
while i<(len(a)+chsiz-1)//chsiz:
    res=embed(a['description'].iloc[i*chsiz:(i+1)*chsiz].to_list())
    all_embs.extend(res)
    print(i,i*chsiz)
    i+=1

0 0
1 16
2 32
3 48
4 64
5 80
6 96
7 112
8 128
9 144
10 160
11 176
12 192
13 208
14 224
15 240
16 256
17 272
18 288
19 304
20 320
21 336
22 352
23 368
24 384
25 400
26 416
27 432
28 448
29 464
30 480
31 496
32 512
33 528
34 544
35 560
36 576
37 592
38 608
39 624
40 640
41 656
42 672
43 688
44 704
45 720
46 736
47 752
48 768
49 784
50 800
51 816
52 832
53 848
54 864
55 880
56 896
57 912
58 928
59 944
60 960
61 976
62 992
63 1008
64 1024
65 1040
66 1056
67 1072
68 1088
69 1104
70 1120
71 1136
72 1152
73 1168
74 1184
75 1200
76 1216
77 1232
78 1248
79 1264
80 1280
81 1296
82 1312
83 1328
84 1344
85 1360
86 1376
87 1392
88 1408
89 1424
90 1440
91 1456
92 1472
93 1488
94 1504
95 1520
96 1536
97 1552
98 1568
99 1584
100 1600
101 1616
102 1632
103 1648
104 1664
105 1680
106 1696
107 1712
108 1728
109 1744
110 1760
111 1776
112 1792
113 1808
114 1824
115 1840
116 1856
117 1872
118 1888
119 1904
120 1920
121 1936
122 1952
123 1968
124 1984
125 2000
126 2016
127 2032
128 2048
129 2064
130 2080
13

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8192 tokens, however you requested 24690 tokens (24690 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [47]:
len(a)

64527

In [56]:
a.iloc[a['description'].apply(len).idxmax()]['description']

'WRENCH,\n21481UFB08,VK2543 SR REC HS CH 155 8,VIK\n21481UFB10,VK2543 SR REC HS CH 155 10,VIK\n21481UFB12,VK2543 SR REC HS CH 155 12,VIK\n21481UFB13,VK2543 SR REC HS CH 155 13,VIK\n21481UFB135,VK2543 SR REC HS CH 155 13.5,VIK\n21481UFB14,VK2543 SR REC HS CH 155 14,VIK\n21481UFB15,VK2543 SR REC HS CH 155 15,VIK\n21481UFB16,VK2543 SR REC HS CH 155 16,VIK\n21481UFB18,VK2543 SR REC HS CH 155 18,VIK\n21481UFB185,VK2543 SR REC HS CH 155 18.5,VIK\n21481UFB19,VK2543 SR REC HS CH 155 19,VIK\n21481UFB195,VK2543 SR REC HS CH 155 19.5,VIK\n21481UFB205,VK2543 SR REC HS CH 155 20.5,VIK\n21481UFB2075,VK2543 SR REC HS CH 155 20.75,VIK\n21481UFB24,VK2543 SR REC HS CH 155 24,VIK\n21481UFB25,VK2543 SR REC HS CH 155 25,VIK\n21481UFB26,VK2543 SR REC HS CH 155 26,VIK\n21481UFB275,VK2543 SR REC HS CH 155 27.5,VIK\n21481UFB2775,VK2543 SR REC HS CH 155 27.75,VIK\n21481UFB2875,VK2543 SR REC HS CH 155 28.75,VIK\n21481UFB29,VK2543 SR REC HS CH 155 29,VIK\n21481UFBY18,VK2543 SR G1REC HS CH155 18,VIK\n21481UFBY185,

In [59]:
a['description'].apply(len).sort_values(ascending=False).iloc[:100]

35147    54469
2524     49678
13641    17707
26553     6096
6676      6017
         ...  
50865      155
53215      155
391        155
42930      152
32858      148
Name: description, Length: 100, dtype: int64

Throw away these three.

In [65]:
c=a[a['description'].apply(len).apply(lambda val:val<10000)].reset_index(drop=True)

In [66]:
chsiz=1024
i=0
all_embs=[]
while i<(len(a)+chsiz-1)//chsiz:
    res=embed(c['description'].iloc[i*chsiz:(i+1)*chsiz].to_list())
    all_embs.extend(res)
    print(i,i*chsiz)
    i+=1

0 0
1 1024
2 2048
3 3072
4 4096
5 5120
6 6144
7 7168
8 8192
9 9216
10 10240
11 11264
12 12288
13 13312
14 14336
15 15360
16 16384
17 17408
18 18432
19 19456
20 20480
21 21504
22 22528
23 23552
24 24576
25 25600
26 26624
27 27648
28 28672
29 29696
30 30720
31 31744
32 32768
33 33792
34 34816
35 35840
36 36864
37 37888
38 38912
39 39936
40 40960
41 41984
42 43008
43 44032
44 45056
45 46080
46 47104
47 48128
48 49152
49 50176
50 51200
51 52224
52 53248
53 54272
54 55296
55 56320
56 57344
57 58368
58 59392
59 60416
60 61440
61 62464
62 63488
63 64512


In [67]:
len(all_embs)==len(c)

True

In [70]:
import json

In [72]:
with open('file.txt','w') as f:
    json.dump(all_embs,f)

In [73]:
!mv file.txt embs.json

In [75]:
len(all_embs)*len(all_embs[0])*4

396435456

396MB.

In [76]:
embs_subset=all_embs[:1000]

In [77]:
with open('embs_subset.json','w') as f:
    json.dump(embs_subset,f)

In [79]:
c.iloc[:1000].to_csv('df_subset.csv',index=0)

In [81]:
c.to_csv('df_full.csv',index=0)

In [84]:
d=' '.join(c['description'].to_list())

In [92]:
#https://stackoverflow.com/questions/13423919/computing-n-grams-using-python
def find_ngrams(input_list, n):
    return [''.join(z) for z in zip(*[input_list[i:] for i in range(n)])]

In [93]:
find_ngrams('abcdefg',3)

['abc', 'bcd', 'cde', 'def', 'efg']

In [94]:
ngrams=[]
for i in range(4,10):
    ngrams.extend(find_ngrams(d,i))

In [97]:
e=pd.Series(ngrams)

In [103]:
list(e.value_counts().sort_values(ascending=False).iloc[:1000].index)

[' VK1',
 ' PD ',
 '5 VK',
 ' QR ',
 '5 VK1',
 ' SR ',
 ' 155',
 ' CH ',
 'REC ',
 '155 ',
 ' REC',
 ' 155 ',
 ' REC ',
 'R RE',
 'R REC',
 'R REC ',
 ' 200',
 'STD ',
 '.5 VK',
 '.5 V',
 '200 ',
 '1/2 ',
 ' WT ',
 ' STD',
 ' STD ',
 '.5 VK1',
 'EC P',
 'C PD',
 'EC PD',
 ' 200 ',
 ' HS ',
 'C PD ',
 'EC PD ',
 'REC P',
 'REC PD',
 ' ASY',
 'D PD',
 'REC PD ',
 'R ST',
 'D PD ',
 'R STD',
 'R STD ',
 'PD C',
 ' 1/2',
 ' PD C',
 'D CH',
 'PD CH',
 ' PD CH',
 'D CH ',
 'PD CH ',
 ' PD CH ',
 'R AS',
 'R ASY',
 'CVR ',
 'CVR AS',
 'VR A',
 'CVR A',
 'VR AS',
 'VR ASY',
 'CVR ASY',
 ' REC P',
 'R REC P',
 ' REC PD',
 'R REC PD',
 'R REC PD ',
 ' REC PD ',
 'CH 1',
 ' BR ',
 ' CH 1',
 ' VK15',
 'VK15',
 ' CVR',
 'ASY,',
 'VK17',
 ' VK17',
 'PD B',
 ' PD B',
 'PLN ',
 'D WT',
 'TD P',
 'D WT ',
 'STD P',
 'PD W',
 ' PD W',
 'TD PD',
 'STD PD',
 ' PLN',
 ' PLN ',
 ' QR R',
 ' QR REC',
 'QR REC',
 ' QR REC ',
 ' QR RE',
 'QR REC ',
 'QR RE',
 'QR R',
 'TD PD ',
 'STD PD ',
 ' GRV',
 'R PL',
 '